## Regression

In [195]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection

In [196]:
from warnings import filterwarnings
filterwarnings('ignore')

In [197]:
stats = pd.read_csv("data_clean.csv")

In [198]:
stats.head()

,link,name,country,country_link,age,overall_rating,potential,value,wage,likes,...,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,player_id,country_id,value_clean,wage_clean,Position
0,https://sofifa.com/player/229558/dayot-upameca...,Dayotchanculle Upamecano,France,/players?na=18,19,77,88,€13M,€18K,96,...,6,7,8,15,9,229558,18,13000000.0,18000.0,LCB
1,https://sofifa.com/player/230767/renato-junior...,Renato Júnior Luz Sanches,Portugal,/players?na=38,20,75,84,€10M,€41K,370,...,5,12,6,13,14,230767,38,10000000.0,41000.0,CM
2,https://sofifa.com/player/237221/juan-foyth/19...,Juan Marcos Foyth,Argentina,/players?na=52,20,74,85,€8.5M,€40K,44,...,8,15,7,14,9,237221,52,8500000.0,40000.0,CB
3,https://sofifa.com/player/228736/wuilker-farin...,Wuilker Faríñez,Venezuela,/players?na=61,20,73,87,€7.5M,€2K,110,...,73,70,69,74,77,228736,61,7500000.0,2000.0,GK
4,https://sofifa.com/player/189509/thiago-alcant...,Thiago Alcântara do Nascimento,Spain,/players?na=45,27,87,87,€51.5M,€140K,109,...,6,11,7,9,13,189509,45,51500000.0,140000.0,CM


In [199]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13758 entries, 0 to 13757
Data columns (total 51 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   link              13758 non-null  object 
 1   name              13758 non-null  object 
 2   country           13758 non-null  object 
 3   country_link      13758 non-null  object 
 4   age               13758 non-null  int64  
 5   overall_rating    13758 non-null  int64  
 6   potential         13758 non-null  int64  
 7   value             13758 non-null  object 
 8   wage              13758 non-null  object 
 9   likes             13758 non-null  int64  
 10  dislikes          13758 non-null  int64  
 11  followers         13758 non-null  int64  
 12  Crossing          13758 non-null  int64  
 13  Finishing         13758 non-null  int64  
 14  Heading Accuracy  13758 non-null  int64  
 15  Short Passing     13758 non-null  int64  
 16  Volleys           13758 non-null  int64 

In [200]:
#gereksiz bilgileri veri setinden silme
stats = stats.drop(["link","name","country","country_link","value","wage","likes","dislikes","followers","player_id","country_id"],axis=1)

In [201]:
stats

,age,overall_rating,potential,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Dribbling,Curve,...,Standing Tackle,Sliding Tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,value_clean,wage_clean,Position
0,19,77,88,47,39,71,72,21,59,32,...,84,75,6,7,8,15,9,13000000.0,18000.0,LCB
1,20,75,84,60,65,64,76,68,77,59,...,70,64,5,12,6,13,14,10000000.0,41000.0,CM
2,20,74,85,41,43,74,73,43,68,41,...,76,72,8,15,7,14,9,8500000.0,40000.0,CB
3,20,73,87,18,19,17,25,15,14,22,...,16,21,73,70,69,74,77,7500000.0,2000.0,GK
4,27,87,87,72,69,54,90,90,90,85,...,67,71,6,11,7,9,13,51500000.0,140000.0,CM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13753,25,62,62,51,63,56,52,66,61,49,...,36,28,13,13,14,14,12,325000.0,1000.0,RM
13754,27,66,67,56,18,70,67,15,51,52,...,64,61,6,13,11,14,10,600000.0,3000.0,RCB
13755,29,68,68,58,29,65,61,39,47,50,...,64,65,8,9,8,11,8,700000.0,3000.0,CB
13756,26,66,66,63,61,51,61,62,72,56,...,27,25,10,10,10,8,8,700000.0,2000.0,LM


In [202]:
#kategorik değişkeni,sayısal değişken yapma / çok sınıflı dönüşüm
from sklearn.preprocessing import LabelEncoder
lbe = LabelEncoder()

In [203]:
stats['Position'] = stats['Position'].astype('str')

In [204]:
stats["Position"] = lbe.fit_transform(df["Position"])

In [205]:
df.head()

,age,overall_rating,potential,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Dribbling,Curve,...,Standing Tackle,Sliding Tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,value_clean,wage_clean,Position
0,19,77,88,47,39,71,72,21,59,32,...,84,75,6,7,8,15,9,13000000.0,18000.0,8
1,20,75,84,60,65,64,76,68,77,59,...,70,64,5,12,6,13,14,10000000.0,41000.0,4
2,20,74,85,41,43,74,73,43,68,41,...,76,72,8,15,7,14,9,8500000.0,40000.0,1
3,20,73,87,18,19,17,25,15,14,22,...,16,21,73,70,69,74,77,7500000.0,2000.0,5
4,27,87,87,72,69,54,90,90,90,85,...,67,71,6,11,7,9,13,51500000.0,140000.0,4


In [206]:
stats.describe()

,age,overall_rating,potential,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Dribbling,Curve,...,Standing Tackle,Sliding Tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,value_clean,wage_clean,Position
count,13758.000000,13758.000000,13758.000000,13758.000000,13758.000000,13758.000000,13758.00000,13758.000000,13758.000000,13758.000000,...,13758.000000,13758.000000,13758.000000,13758.000000,13758.000000,13758.000000,13758.000000,1.375800e+04,13758.000000,13758.00000
mean,24.802951,67.772060,72.641736,51.666376,47.552188,53.721980,60.60125,44.813272,57.660343,49.414304,...,48.737753,46.515627,16.257523,16.039541,15.887266,16.085405,16.339730,3.155406e+06,12151.314595,11.69763
std,4.517843,6.725034,6.035821,18.305495,19.563786,17.270443,14.26085,17.881840,18.593676,18.526443,...,21.851820,21.569739,17.579222,16.762707,16.351868,16.969712,17.879968,6.360112e+06,24371.218714,8.81902
min,16.000000,46.000000,46.000000,6.000000,2.000000,4.000000,7.00000,4.000000,4.000000,6.000000,...,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+04,1000.000000,0.00000
25%,21.000000,63.000000,69.000000,40.000000,32.000000,46.000000,56.00000,32.000000,52.000000,37.000000,...,28.000000,25.000000,8.000000,8.000000,8.000000,8.000000,8.000000,4.750000e+05,2000.000000,4.00000
50%,24.000000,68.000000,72.000000,56.000000,52.000000,57.000000,64.00000,46.000000,63.000000,52.000000,...,56.000000,53.000000,11.000000,11.000000,11.000000,11.000000,11.000000,9.500000e+05,4000.000000,9.00000
75%,28.000000,72.000000,76.000000,65.000000,63.000000,65.000000,70.00000,59.000000,70.000000,64.000000,...,67.000000,65.000000,14.000000,14.000000,14.000000,14.000000,14.000000,3.200000e+06,13000.000000,19.00000
max,45.000000,94.000000,95.000000,93.000000,95.000000,94.000000,93.00000,90.000000,97.000000,94.000000,...,93.000000,90.000000,90.000000,92.000000,92.000000,91.000000,93.000000,1.105000e+08,565000.000000,27.00000


In [238]:
df = stats.copy()
df = df.dropna()
df.head()

,age,overall_rating,potential,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Dribbling,Curve,...,Standing Tackle,Sliding Tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,value_clean,wage_clean,Position
0,19,77,88,47,39,71,72,21,59,32,...,84,75,6,7,8,15,9,13000000.0,18000.0,8
1,20,75,84,60,65,64,76,68,77,59,...,70,64,5,12,6,13,14,10000000.0,41000.0,4
2,20,74,85,41,43,74,73,43,68,41,...,76,72,8,15,7,14,9,8500000.0,40000.0,1
3,20,73,87,18,19,17,25,15,14,22,...,16,21,73,70,69,74,77,7500000.0,2000.0,5
4,27,87,87,72,69,54,90,90,90,85,...,67,71,6,11,7,9,13,51500000.0,140000.0,4


In [239]:
#aykırı değer yakalama
from sklearn.neighbors import LocalOutlierFactor

In [240]:
clf = LocalOutlierFactor(n_neighbors = 20, contamination = 0.1)

In [241]:
clf.fit_predict(df)

array([1, 1, 1, ..., 1, 1, 1])

In [242]:
df_scores = clf.negative_outlier_factor_

In [243]:
np.sort(df_scores)[0:20]

array([-155.02141627, -154.89230092, -152.02418237, -138.69118866,
       -137.19851522,  -94.11734536,  -92.10552348,  -91.15886341,
        -90.84318375,  -75.2180578 ,  -73.2930637 ,  -70.19780301,
        -62.26767127,  -61.85580327,  -61.79944747,  -60.94990171,
        -60.42817482,  -59.49819612,  -56.96730946,  -56.85167795])

In [244]:
len(df_scores)

13758

In [245]:
esik_deger = np.sort(df_scores)[1000]

In [246]:
aykiri_tf = df_scores > esik_deger

In [247]:
aykiri_tf

array([ True,  True,  True, ...,  True,  True,  True])

In [248]:
#aykırı değerleri veri setinden silme
yeni_df  = df[df_scores > esik_deger]

In [249]:
yeni_df

,age,overall_rating,potential,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Dribbling,Curve,...,Standing Tackle,Sliding Tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,value_clean,wage_clean,Position
0,19,77,88,47,39,71,72,21,59,32,...,84,75,6,7,8,15,9,13000000.0,18000.0,8
1,20,75,84,60,65,64,76,68,77,59,...,70,64,5,12,6,13,14,10000000.0,41000.0,4
2,20,74,85,41,43,74,73,43,68,41,...,76,72,8,15,7,14,9,8500000.0,40000.0,1
3,20,73,87,18,19,17,25,15,14,22,...,16,21,73,70,69,74,77,7500000.0,2000.0,5
4,27,87,87,72,69,54,90,90,90,85,...,67,71,6,11,7,9,13,51500000.0,140000.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13753,25,62,62,51,63,56,52,66,61,49,...,36,28,13,13,14,14,12,325000.0,1000.0,22
13754,27,66,67,56,18,70,67,15,51,52,...,64,61,6,13,11,14,10,600000.0,3000.0,18
13755,29,68,68,58,29,65,61,39,47,50,...,64,65,8,9,8,11,8,700000.0,3000.0,1
13756,26,66,66,63,61,51,61,62,72,56,...,27,25,10,10,10,8,8,700000.0,2000.0,12


In [250]:
#silinen değerler
df[df_scores < esik_deger]

,age,overall_rating,potential,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Dribbling,Curve,...,Standing Tackle,Sliding Tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,value_clean,wage_clean,Position
13,20,70,84,65,70,43,69,51,74,56,...,41,37,11,6,13,11,8,1.849073e+06,9103.854390,0
19,18,67,82,48,33,52,69,33,56,59,...,64,57,11,8,12,9,15,1.400000e+06,1000.000000,2
88,32,81,81,86,62,66,84,76,76,81,...,80,79,1,1,1,1,1,1.350000e+07,120000.000000,4
90,33,76,76,44,79,90,73,81,67,57,...,25,15,2,2,3,4,2,4.800000e+06,76000.000000,26
104,18,66,82,54,55,34,68,48,68,58,...,45,41,12,11,8,12,6,1.300000e+06,7000.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13702,26,62,63,62,52,55,65,49,66,41,...,55,49,15,8,11,7,8,3.500000e+05,7000.000000,4
13727,26,68,71,37,24,70,58,34,55,28,...,70,64,8,16,14,8,15,9.250000e+05,5000.000000,1
13746,23,55,60,34,57,57,45,46,49,37,...,18,12,10,10,15,15,8,1.200000e+05,2000.000000,26
13748,28,66,66,64,25,63,62,30,57,35,...,65,66,9,6,12,7,13,7.413704e+05,3717.492984,18


In [251]:
yeni_df.describe()

,age,overall_rating,potential,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Dribbling,Curve,...,Standing Tackle,Sliding Tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,value_clean,wage_clean,Position
count,12757.000000,12757.000000,12757.000000,12757.000000,12757.000000,12757.000000,12757.000000,12757.000000,12757.000000,12757.000000,...,12757.000000,12757.000000,12757.00000,12757.000000,12757.000000,12757.000000,12757.000000,1.275700e+04,12757.000000,12757.000000
mean,24.660814,67.989026,72.921376,51.919260,47.817826,53.871522,60.902563,45.028063,58.008858,49.709414,...,48.856001,46.600690,16.16101,15.939171,15.791409,15.976483,16.231873,3.291140e+06,12139.747564,11.707141
std,4.387092,6.676027,5.931313,18.280408,19.568957,17.212396,14.165708,17.891970,18.542350,18.510698,...,21.846539,21.568341,17.47438,16.654041,16.265808,16.845081,17.774016,6.419010e+06,24067.989512,8.803212
min,16.000000,47.000000,48.000000,6.000000,2.000000,4.000000,7.000000,4.000000,4.000000,6.000000,...,2.000000,4.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000e+04,1000.000000,0.000000
25%,21.000000,64.000000,69.000000,41.000000,32.000000,46.000000,56.000000,32.000000,53.000000,37.000000,...,28.000000,25.000000,8.00000,8.000000,8.000000,8.000000,8.000000,5.000000e+05,2000.000000,4.000000
50%,24.000000,68.000000,73.000000,57.000000,52.000000,57.000000,64.000000,47.000000,63.000000,52.000000,...,56.000000,53.000000,11.00000,11.000000,11.000000,11.000000,11.000000,1.000000e+06,4000.000000,9.000000
75%,28.000000,72.000000,77.000000,66.000000,64.000000,66.000000,70.000000,59.000000,70.000000,64.000000,...,67.000000,65.000000,14.00000,14.000000,14.000000,14.000000,14.000000,3.300000e+06,13000.000000,19.000000
max,40.000000,94.000000,95.000000,93.000000,94.000000,94.000000,93.000000,90.000000,95.000000,94.000000,...,93.000000,90.000000,90.00000,92.000000,92.000000,90.000000,93.000000,9.650000e+07,455000.000000,27.000000


In [252]:
dms = pd.get_dummies(yeni_df['Position'])
y = yeni_df["wage_clean"] #bağımlı değişken
X_ = yeni_df.drop(['wage_clean', 'Position'], axis=1).astype('float64') #bağımsız değişkenin çıkarılması
X = pd.concat([X_, dms], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.25,
                                                    random_state=42)

## Wage pred.

In [19]:
!pip install xgboost

  Using cached xgboost-1.3.3-py3-none-win_amd64.whl (95.2 MB)


In [253]:
import xgboost

In [254]:
from xgboost import XGBRegressor

In [255]:
xgb = XGBRegressor().fit(X_train,y_train)

In [256]:
y_pred=xgb.predict(X_test)

In [257]:
np.sqrt(mean_squared_error(y_test, y_pred))

12152.338982582922

In [258]:
xgb_params = {"learning_rate": [0.1,0.01,0.03],
             "max_depth": [2,3,4,5,8],
             "n_estimators": [50,100,250,500],
             "colsample_bytree": [0.4,0.7,1]}

In [185]:
xgb_cv_model = GridSearchCV(xgb,xgb_params, cv = 10, n_jobs = -1, verbose = 2).fit(X_train, y_train)

Fitting 10 folds for each of 180 candidates, totalling 1800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed: 16.5min
[Parallel(n_jobs=-1)]: Done 1800 out of 1800 | elapsed: 22.4min finished


In [186]:
xgb_cv_model.best_params_

{'colsample_bytree': 1,
 'learning_rate': 0.03,
 'max_depth': 3,
 'n_estimators': 250}

In [187]:
xgb_tuned = XGBRegressor(colsample_bytree = 1, 
                         learning_rate = 0.03, 
                         max_depth = 2, 
                         n_estimators = 500).fit(X_train, y_train)

In [188]:
y_pred = xgb_tuned.predict(X_test)

In [189]:
np.sqrt(mean_squared_error(y_test, y_pred))

12312.401946953223